In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, date_format, when, udf
from haversine import haversine
from pyspark.sql.types import DoubleType

In [0]:
df = spark.read.table("fordgobike_tripdata_3_csv")

In [0]:
df = df.withColumn("start_time_timestamp", to_timestamp("start_time", "mm:ss.S"))
df = df.withColumn("start_time_timestamp", date_format("start_time_timestamp", "mm:ss.S"))
df = df.withColumn("end_time_timestamp", to_timestamp("end_time", "mm:ss.S"))
df = df.withColumn("end_time_timestamp", date_format("end_time_timestamp", "mm:ss.S"))
df = df.drop("start_time", "end_time")

In [0]:
df = df.withColumn("start_time", when(df.start_time_timestamp < df.end_time_timestamp, df.start_time_timestamp).otherwise(df.end_time_timestamp))
df = df.withColumn("end_time", when(df.start_time_timestamp < df.end_time_timestamp, df.end_time_timestamp).otherwise(df.start_time_timestamp))
df = df.drop("start_time_timestamp", "end_time_timestamp")
df.show()

+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+----------+--------+
|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|start_time|end_time|
+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+----------+--------+
|              74|Laguna St at Haye...|           37.77643482|            -122.426244|            43|San Francisco Pub...|          37.7787677|         -122.4159292|     96|  Customer|             1987|         Male|credit card|   12:50.2| 57:39.7|
|   

In [0]:
@udf(returnType=DoubleType())
def haversine_distance(lat1, lon1, lat2, lon2):
    start = (lat1, lon1)
    end = (lat2, lon2)
    distance = haversine(start, end)
    return distance

# Apply the user-defined function to the DataFrame
df = df.withColumn("distance", haversine_distance(df.start_station_latitude, df.start_station_longitude, df.end_station_latitude, df.end_station_longitude))
df.show()

+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+----------+--------+------------------+
|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|start_time|end_time|          distance|
+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+----------+--------+------------------+
|              74|Laguna St at Haye...|           37.77643482|            -122.426244|            43|San Francisco Pub...|          37.7787677|         -122.4159292|     96|  Customer|            